In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import base64
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps


from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "aacuser"
auth_source = 'AAC'
shelter = AnimalShelter(username, password, auth_src_db=auth_source)

# class read method must support return of cursor object
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash("AACDashboard")

image_filename = "grazioso_salvare_logo.png"
encoded_image = base64.b64encode(open(image_filename, "rb").read())

app.layout = html.Div([
    html.Div(
        className="row",
        style={"display": "flex"},
        children=[ html.Img(src="data:image/png;base64,{}".format(encoded_image.decode()), 
                            alt="Grazioso Salvare logo", 
                            height="150px"),
                  html.Div(children=html.B(html.H1("SNHU CS-340 Dashboard")), 
                           style={"display": "flex", "justify-content": "center", "align-items": "center", "width": "100%"}) ]
    ),
    html.Hr(),
    dcc.Dropdown(
        id="filter_radioitems",
        options=[
            { "label": "All", "value": "reset" },
            { "label": "Water Rescue", "value": "wtr_resc" },
            { "label": "Mountain or Wilderness Rescue", "value": "mtn_wldn_resc" },
            { "label": "Disaster Rescue or Individual Tracking", "value": "dis_resc_indiv_trkg" }
        ],
        value="wtr_resc",
        clearable=False
    ),
    html.Hr(),
    dt.DataTable(
        id="datatable-id",
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict("records"),
        page_action="native",
        page_current= 0,
        page_size= 10,
        row_deletable=False,
        row_selectable=False,
        sort_action="native",
        sort_mode="multi"
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className="row",
         style={"display": "flex"},
         children=[
            html.Div(
                id="graph-id",
                className="col s12 m6"
            ),
            html.Div(
                id="map-id",
                className="col s12 m6"
            )
        ]
     ),
    html.Br(),
    html.Footer(
        html.Center(html.I("Eric Slutz CS-340 Project 2"))
    )
])


#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output("datatable-id","data"),
              [Input("filter_radioitems", "value")])
def update_dashboard(filter):
    if filter == "wtr_resc":
        query = { "breed": { "$in": [ "Labrador Retriever Mix", 
                                   "Chesapeake Bay Retriever", 
                                   "Newfoundland" ] 
                           }, 
                "sex_upon_outcome": "Intact Female", 
                "$and": [ { "age_upon_outcome_in_weeks": { "$gte": 26 } }, 
                       { "age_upon_outcome_in_weeks": { "$lte": 156 } } ] }
    elif filter == "mtn_wldn_resc":
        query = { "breed": { "$in": [ "German Shepherd", 
                                   "Alaskan Malamute", 
                                   "Old English Sheepdog", 
                                   "Siberian Husky", 
                                   "Rottweiler" ] 
                           }, 
                "sex_upon_outcome": "Intact Male", 
                "$and": [ { "age_upon_outcome_in_weeks": { "$gte": 26 } }, 
                       { "age_upon_outcome_in_weeks": { "$lte": 156 } } ] }
    elif filter == "dis_resc_indiv_trkg":
        query = { "breed": { "$in": [ "Doberman Pinscher", 
                                   "German Shepherd", 
                                   "Golden Retriever", 
                                   "Bloodhound", 
                                   "Rottweiler" ] 
                           }, 
                "sex_upon_outcome": "Intact Male", 
                "$and": [ { "age_upon_outcome_in_weeks": { "$gte": 20 } }, 
                       { "age_upon_outcome_in_weeks": { "$lte": 300 } } ] }
    else:
        query = {}

    df = pd.DataFrame(list(shelter.read(query)))
    data = df.to_dict("records")

    return data


@app.callback(
    Output("datatable-id", "style_data_conditional"),
    [Input("datatable-id", "selected_columns")]
)
def update_styles(selected_columns):
    return [{
        "if": { "column_id": i },
        "background_color": "#D2F3FF"
    } for i in selected_columns]


@app.callback(
    Output("graph-id", "children"),
    [Input("datatable-id", "derived_viewport_data")])
def update_graphs(view_data):
    dff = pd.DataFrame.from_dict(view_data)
    return [
       dcc.Graph(
           style={"width": "400px", "height": "400px"},
           figure = px.pie(dff, names="breed")
       )
    ]


@app.callback(
    Output("map-id", "children"),
    [Input("datatable-id", "derived_viewport_data")])
def update_map(view_data):
    dff = pd.DataFrame.from_dict(view_data)
    lats = dff["location_lat"].to_string().split()
    longs = dff["location_long"].to_string().split()
    marker_locations = []
    for i in range(1, len(lats)):
        marker_locations.append([float(lats[i]), float(longs[i])])     
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(
            style={"width": "400px", "height": "400px"},
            center=[30.75,-97.48],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                dl.Marker(
                    position=[30.75,-97.48],
                    children=[
                        dl.Tooltip(dff["breed"]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff["name"])
                        ])
                    ]
                )
            ]
        )
    ]


app

/var/folders/32/2rwdgx8j25591ywgkvm2nt000000gn/T/ipykernel_99035/1593662393.py:5: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

/var/folders/32/2rwdgx8j25591ywgkvm2nt000000gn/T/ipykernel_99035/1593662393.py:6: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`

/var/folders/32/2rwdgx8j25591ywgkvm2nt000000gn/T/ipykernel_99035/1593662393.py:8: UserWarning:


The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`

